In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)


from datetime import date

today = str(date.today())
end_date = today

import glob

path = r'data/final/'  # use your path
allFiles = glob.glob(path + "/*.csv")

dftrain = pd.DataFrame()

for _file in allFiles[:-2]:
        try:
            t = pd.read_csv(_file,header=0)
            if len(t[(t["fvalue"].isnull())]) > 0:
                print("Some nulls in " + _file)
            dftrain = pd.concat([dftrain,t])
        except:
            pass
print("Last file loaded: " + _file)
test_file = allFiles[-2:-1][0]
print("Test file " + test_file)
dfTest = pd.read_csv(test_file,header=0)

predict_file = 'data/final/2017-01-26.csv'
print("Predict file " + predict_file)
dfPredict = pd.read_csv(predict_file, header=0)

Last file loaded: data/final/2017-01-24.csv
Test file data/final/2017-01-25.csv
Predict file data/final/2017-01-26.csv


In [3]:
dftrain[features].describe()

NameError: name 'features' is not defined

In [4]:
df_train = dftrain

scaler = StandardScaler()
features = ['Salary','AvgPointsPerGame'
    ,'7GameAvg','FloorAvg'
    ,'4GameAvg','value','fvalue'
    ,'atHome','LastGame','O/U','odds',"dvp"]

In [5]:
X_train = scaler.fit_transform(df_train[features].values)
y_train = df_train["Final"].values
#y_train_onehot = pd.get_dummies(df_train['4GameAvg']).values

In [6]:
df_test = dfTest

X_test = scaler.transform(df_test[features].values)
y_test = df_test["Final"].values

In [29]:
from sklearn.ensemble import RandomForestRegressor

modelRFR = RandomForestRegressor(n_estimators=92,
                              random_state=1,
                              verbose=False,
                              warm_start=False)
modelRFR = modelRFR.fit(X_train, y_train)

#dfPredict = scaler.transform(X_test[features].values)
#X_test = scaler.transform(df_test[features].values)
y_tree_prediction = modelRFR.predict(X_test)
print ("\naccuracy", np.sum(abs(y_tree_prediction-y_test)<5) / float(len(y_test)))


accuracy 0.867857142857


In [8]:
#y_tree_prediction
#pd.concat([dfTest["Name"],pd.DataFrame(y_tree_prediction),pd.DataFrame(y_test)],axis=1)

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Activation

start = time()

model = Sequential()
model.add(Dense(12, input_dim=len(features), init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
          
          
model.compile(optimizer='adam', metrics=['accuracy'],loss="mean_squared_error")

model.fit(X_train, y_train,
          validation_data=(X_test,y_test),
          verbose=False,
          nb_epoch=150, batch_size=10)

print ('\ntime taken %s seconds' % str(time() - start))


time taken 180.73180508613586 seconds



time taken 183.63301515579224 seconds


In [11]:
scores = model.evaluate(X_test,y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

 32/280 [==>...........................] - ETA: 0s

acc: 5.71%


In [54]:
#dfpredict = pd.read_csv('data/output/3017-01-21.csv', header=0)

#X_test = scaler.transform(dfPredict1[features].values)

y_predict = model.predict(X_test)
#pd.DataFrame(y_predict).describe()
# (y_predict - y_test) < 10
print(np.sum(abs(y_predict.T[0]-y_test) < 5)/float(len(y_test)))
#y_predict[[range(0,len(y_predict))]]
#print ("\naccuracy", np.sum(abs(y_predict-y_test)<10) / float(len(y_test)))
#y_tree_predict = modelRFR.predict(X_test)

0.842857142857


array([[ 4.79201948,  4.00939974,  3.70397966, ...,  0.32351783,
        -0.21942942,  1.40388233],
       [ 4.5095109 ,  3.68334913,  3.96535331, ...,  0.81751411,
        -0.5786073 ,  0.4444856 ],
       [ 3.66198516,  3.0093542 ,  3.13647773, ...,  0.30375798,
         1.72013115, -0.54489227],
       ..., 
       [-0.7451487 , -0.78523642, -0.64553744, ...,  0.46183679,
        -1.22512749,  1.34392003],
       [-0.7451487 , -0.49695464, -0.51185588, ...,  0.46183679,
        -1.22512749,  1.7636561 ],
       [-0.7451487 , -0.81192756, -0.65743318, ...,  0.46183679,
         1.21728211,  1.52380692]])

In [22]:
output = pd.concat([dfTest,
            pd.DataFrame(y_predict,columns=["NN"]),
            pd.DataFrame(y_tree_prediction,columns=["RF"])]
          ,axis=1)
#output.to_csv('data/output/2017-01-24.csv', sep=',', encoding='utf-8', index=False, float_format='%.3f')

In [23]:
output.loc[:,("Name","NN","RF","AvgPointsPerGame","Final","O/U","odds","dvp")].sort_values(by=["O/U","NN","dvp"], ascending=False)

,Name,NN,RF,AvgPointsPerGame,Final,O/U,odds,dvp
7,Isaiah Thomas,55.215801,56.937500,43.813,60.25,229.0,4.0,9.50
1,James Harden,53.166351,47.100543,60.651,58.50,229.0,-4.0,19.00
30,Al Horford,43.038368,45.725543,36.140,47.75,229.0,4.0,22.00
49,Jae Crowder,41.257439,42.086957,26.403,45.00,229.0,4.0,25.00
166,Nene Hilario,30.168289,32.241848,15.976,0.00,229.0,-4.0,21.00
79,Ryan Anderson,26.764179,29.236413,23.778,29.25,229.0,-4.0,25.00
41,Marcus Smart,22.616669,23.970109,24.970,26.00,229.0,4.0,18.00
116,Sam Dekker,22.381374,23.989130,15.932,24.50,229.0,-4.0,19.75
91,Clint Capela,21.664978,24.260870,25.826,25.75,229.0,-4.0,21.00
58,Patrick Beverley,19.186241,21.437500,27.824,22.50,229.0,-4.0,24.00


In [161]:
dfPredict1["NN2"]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
      ... 
130    0.0
131    0.0
132    0.0
133    0.0
134    0.0
135    0.0
136    0.0
137    0.0
138    0.0
139    0.0
140    0.0
141    0.0
142    0.0
143    0.0
144    0.0
145    0.0
146    0.0
147    0.0
148    0.0
149    0.0
150    0.0
151    0.0
152    0.0
153    0.0
154    0.0
155    0.0
156    0.0
157    0.0
158    0.0
159    0.0
Name: NN2, dtype: float64